# 4.2 データ整形

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

In [ ]:
# 追加ライブラリのimport
import seaborn as sns

# サンプルデータの読み込み
df_titanic = sns.load_dataset("titanic")

# 項目名の日本語化
columns_t = ['生存', '等室', '性別', '年齢', '兄弟配偶者数', 
             '両親子供数', '料金', '乗船港コード', '等室名', 
             '男女子供', '成人男子', 'デッキ', '乗船港', '生存可否', '独身']
df_titanic.columns = columns_t

#### データ確認

In [ ]:
display(df_titanic.head())

In [ ]:
print(df_titanic.shape)

### 4.2.1 不要な項目の削除

In [ ]:
# 余分な列削除

# 「等室名」 (「等室」と同じ)
df1 = df_titanic.drop('等室名', axis=1)

# 「乗船港」 (「乗船港コード」と同じ)
df2 = df1.drop('乗船港', axis=1)

# 「生存可否」 (「生存」と同じ)
df3 = df2.drop('生存可否', axis=1)

# 結果確認
display(df3.head())

### 4.2.2 欠損値の対応

In [ ]:
# 欠損値確認
display(df3.isnull().sum())

In [ ]:
display(df3['デッキ'].value_counts())

#### 方針の決定

**乗船港コード**: 件数が2件と少ない
-> 行ごと削除する      

**年齢**: 数値データであり、欠損行数が177件とかなり多い
-> 他データの平均値で代用  

**デッキ**: コード値データであり、欠損行数が688行と相当多い
-> 欠損を意味するダミーコードを振って全行処理対象とする

In [ ]:
# 乗船港コード: 件数が2件と少ない 
# -> 行ごと削除する

# dropna関数を利用する
df4 = df3.dropna(subset = ['乗船港コード'])

# 年齢: 数値データであり欠損行数が177件とかなり多い 
# -> 他データの平均値で代用

# 平均値の計算
age_average = df4['年齢'].mean()

# fillna関数の利用
df5 = df4.fillna({'年齢': age_average})

# デッキ: コード値データであり欠損行数が688行と相当多い
# -> 欠損を意味するダミーコードを振って全行処理対象とする

# replace関数の利用  (ダミーコードは 'N' とする)
df6 = df5.replace({'デッキ': {np.nan: 'N'}})

In [ ]:
# 結果確認
display(df6.isnull().sum())

display(df6.head())

### 4.2.3 2値ラベルの数値化

**性別**: male / female  
**成人男子**: True / False  
**独身**: True / False  
    
をそれぞれ 1/ 0 に数値化する


#### 「性別」の数値化

In [ ]:
display(df6['性別'].value_counts())

In [ ]:
# 辞書 mf_map の定義
mf_map = {'male': 1, 'female': 0}

# map関数を利用して数値化
df7 = df6.copy()
df7['性別'] = df7['性別'].map(mf_map)

# 結果確認
display(df7.head())

#### 「成人男子」「独身」の数値化

In [ ]:
display(df7['成人男子'].value_counts())

In [ ]:
# 辞書 tf_map の定義
tf_map = {True: 1, False: 0}

# map関数を利用して数値化
df8 = df7.copy()
df8['成人男子'] = df8['成人男子'].map(tf_map)

# map関数を利用して数値化
df9 = df8.copy()
df9['独身'] = df8['独身'].map(tf_map)

# 結果確認
display(df9.head())

### 4.2.4 多値ラベルの数値化 
One-Hot エンコーディング

In [ ]:
# 変換前
display(df9[['男女子供']].head(10))

In [ ]:
# get_dummies関数の利用サンプル

w = pd.get_dummies(df9['男女子供'], prefix='男女子供')
display(w.head(10))

In [ ]:
# get_dummies関数でカテゴリ値をone hot vectorに展開する関数を定義
# df 対象データフレーム
# column 対象列

def enc(df, column):
    # One Hot Vector生成
    df_dummy = pd.get_dummies(df[column], prefix=column)
    # 元列の削除
    df_drop = df.drop([column], axis=1)
    # 削除したデータフレームと、One Hot生成列を連結
    df1 = pd.concat([df_drop,df_dummy],axis=1)
    return df1

#### 男女子供
man / woman / child

In [ ]:
# 項目値の確認
display(df9['男女子供'].value_counts())

In [ ]:
# One-Hot エンコード化

# 男女子供,
df10 = enc(df9, '男女子供')

# 結果確認
display(df10.head())

#### 乗船港コードとデッキ

In [ ]:
# One-Hot エンコード化

# 乗船港コード
df11 = enc(df10, '乗船港コード')

# デッキ
df12 = enc(df11, 'デッキ')

# 結果確認
display(df12.head())

### 4.2.5 正規化

In [ ]:
# standardization 

df13 = df12.copy()
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
df13[['年齢', '料金']] = stdsc.fit_transform(df13[['年齢', '料金']])

# 結果確認
display(df13.head())